In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
%config IPCompleter.greedy=True
%config Completer.use_jedi = False

In [ ]:
# to make display(...) show all rows of a dataframe
pd.set_option('display.max_rows', None)

In [ ]:
pd.reset_option('display.max_rows')

In [ ]:
results

In [ ]:
# evaluate results file 
results = pd.read_csv('results.csv')

def rate(qresults, ks):
    qmovie_id = qresults.movieId.head(1)    
    rating = pd.DataFrame(index=[qmovie_id])
    for k in ks:
        rmovie_ids = [int(y.split("_")[0]) for y in qresults.movieclipId.head(k)]
        # rate relevance    
        qgenres = genres.loc[qmovie_id]
        rgenres = genres.loc[rmovie_ids]
        rjscore = rgenres.apply(lambda row: jaccard_score(qgenres.values[0], row), axis=1)
        prec = len(rjscore[rjscore > 0.5]) / len(rjscore)
        rating['P@' + str(k)] = prec
        # rate diversity
        qtags = tags.loc[qmovie_id].iloc[0]
        qtags = qtags[qtags > 0].index
        rtags = (tags.loc[rmovie_ids] > 0).any()
        rtags = rtags[rtags == True].index
        abs_tag_cov = len(rtags)
        rel_tag_cov = len(rtags) / len(qtags)
        rating['AbsTagCov@' + str(k)] = abs_tag_cov
        rating['RelTagCov@' + str(k)] = rel_tag_cov
        # TODO not sure how to calculate tag entropy...
    return rating
    
results_per_query = results.groupby('movieId')
# TODO this is super slow...
ratings = results_per_query.apply(lambda x: rate(x, [10]))
ratings.index = ratings.index.get_level_values(0)
display(ratings)

In [ ]:
# parse data
movies = pd.read_csv('MMSR_dataset_2019/devset_movies.csv')
clips = pd.read_csv('MMSR_dataset_2019/devset_ids.csv')
tags = pd.read_csv('MMSR_dataset_2019/features/Metadata/devset_TagFeatures.csv')
genres = pd.read_csv('MMSR_dataset_2019/features/Metadata/devset_GenreFeatures.csv')

# index by movieId
movies.set_index('movieId', inplace=True)
tags.set_index('movieId', inplace=True)
clips.set_index('movieId', inplace=True)
genres.set_index('movieId', inplace=True)

# fix missing movies in tags
tags = movies.join(tags).drop('title', axis=1).fillna(0)

In [ ]:
# compute cosine similarities between tag tf_idf vectors
tags_cos_sim = pd.DataFrame(cosine_similarity(tags.values),
                            columns=tags.index.values, 
                            index=tags.index)

In [ ]:
# generate results file (but do retrieval only based on tfidf features)
results = pd.DataFrame(columns=['movieId', 'movieclipId', 'sim', 'rank'])

for movieId in movies.index:
    sim = tags_cos_sim[[movieId]]
    sim = sim.reset_index()
    sim = sim.rename(columns={movieId : 'sim', 'movieId': 'otherMovieId'})
    sim = sim.join(clips, on='otherMovieId', how='outer')
    sim = sim.reset_index(drop=True)
    sim = sim.drop('otherMovieId', axis=1)
    sim = sim.sort_values('sim', ascending=False)
    sim = sim.head(100)
    sim['rank'] = np.arange(100)
    sim['movieId'] = np.full(100, movieId)
    results = pd.concat([results, sim], ignore_index=True)
results.to_csv('results.csv', columns=['movieId', 'movieclipId', 'sim', 'rank'], index=False)

